In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow import keras
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

In [3]:
# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config = config)

In [4]:
shakespeare_url = 'https://homl.info/shakespeare'
filpath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filpath) as f:
    shakespeare_text = f.read()

### 전처리 (데이터셋 준비)

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [6]:
tokenizer.texts_to_sequences(['first'])

[[20, 6, 9, 8, 3]]

In [7]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [8]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [9]:
max_id, dataset_size

(39, 1115394)

In [10]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [11]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
train_size

1003854

In [12]:
n_steps = 100

window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [13]:
print(dataset)

<WindowDataset shapes: DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([])), types: DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([]))>


In [14]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [15]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [16]:
dataset = dataset.map(lambda x_batch, y_batch : (tf.one_hot(x_batch, depth=max_id), y_batch))

In [17]:
dataset = dataset.prefetch(1)

### CHAR RNN model create training

In [18]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = 'adam')

In [20]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)

Epoch 1/10
   3623/Unknown - 32s 9ms/step - loss: 1.5555

KeyboardInterrupt: 